In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv(r"/content/uber_rides_data.xlsx - sample_train.csv")

In [3]:
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [4]:
df.shape

(200000, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


In [6]:
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'])

In [7]:
df = df.dropna(axis=0)

In [8]:
df.isnull().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [9]:
df.describe()

,ride_id,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.999990e+05,199999.000000,199999.000000,199999.000000,199999.000000,199999.000000,199999.000000
mean,2.771248e+07,11.359892,-72.527631,39.935881,-72.525292,39.923890,1.684543
std,1.601386e+07,9.901760,11.437815,7.720558,13.117408,6.794829,1.385995
min,1.000000e+00,-52.000000,-1340.648410,-74.015515,-3356.666300,-881.985513,0.000000
25%,1.382534e+07,6.000000,-73.992065,40.734796,-73.991407,40.733823,1.000000
50%,2.774524e+07,8.500000,-73.981823,40.752592,-73.980093,40.753042,1.000000
75%,4.155535e+07,12.500000,-73.967154,40.767158,-73.963659,40.768001,2.000000
max,5.542357e+07,499.000000,57.418457,1644.421482,1153.572603,872.697628,208.000000


In [10]:
import math
import numpy as np


R = 6371

# Convert latitude and longitude from degrees to radians
df['pickup_lat_rad'] = np.radians(df['pickup_latitude'])
df['pickup_lon_rad'] = np.radians(df['pickup_longitude'])
df['dropoff_lat_rad'] = np.radians(df['dropoff_latitude'])
df['dropoff_lon_rad'] = np.radians(df['dropoff_longitude'])

# Calculate differences in latitude and longitude
df['dlat'] = df['dropoff_lat_rad'] - df['pickup_lat_rad']
df['dlon'] = df['dropoff_lon_rad'] - df['pickup_lon_rad']

# Haversine formula
df['a'] = np.sin(df['dlat'] / 2)**2 + np.cos(df['pickup_lat_rad']) * np.cos(df['dropoff_lat_rad']) * np.sin(df['dlon'] / 2)**2
df['c'] = 2 * np.arctan2(np.sqrt(df['a']), np.sqrt(1 - df['a']))

# Calculate distance and assign it to a new column
df['haversine_dist'] = R * df['c']

# Calculate the median Haversine distance
median_distance = np.median(df['haversine_dist'])

print("Median Haversine Distance:", median_distance, "kilometers")

df.drop(['pickup_lat_rad', 'pickup_lon_rad', 'dropoff_lat_rad', 'dropoff_lon_rad', 'dlat', 'dlon', 'a', 'c'], axis=1, inplace=True)


Median Haversine Distance: 2.1209923961833708 kilometers


In [11]:
zero_distance_ride = df[df['haversine_dist'] == 0]
print(len(zero_distance_ride))

5632


In [12]:
mean_fare_for_zero_distance_rides = zero_distance_ride['fare_amount'].mean()

print("Mean 'fare_amount' for rides with 0.0 Haversine Distance:",
      mean_fare_for_zero_distance_rides)

Mean 'fare_amount' for rides with 0.0 Haversine Distance: 11.585317826704546


In [13]:
import numpy as np




R = 6371

# Convert latitude and longitude from degrees to radians
df['pickup_lat_rad'] = np.radians(df['pickup_latitude'])
df['pickup_lon_rad'] = np.radians(df['pickup_longitude'])
df['dropoff_lat_rad'] = np.radians(df['dropoff_latitude'])
df['dropoff_lon_rad'] = np.radians(df['dropoff_longitude'])

# Calculate differences in latitude and longitude
df['dlat'] = df['dropoff_lat_rad'] - df['pickup_lat_rad']
df['dlon'] = df['dropoff_lon_rad'] - df['pickup_lon_rad']

# Haversine formula
df['a'] = np.sin(df['dlat'] / 2)**2 + np.cos(df['pickup_lat_rad']) * np.cos(df['dropoff_lat_rad']) * np.sin(df['dlon'] / 2)**2
df['c'] = 2 * np.arctan2(np.sqrt(df['a']), np.sqrt(1 - df['a']))

# Calculate distance and assign it to a new column
df['haversine_dist'] = R * df['c']


costliest_ride = df[df['fare_amount'] == df['fare_amount'].max()]

haversine_dist_costliest_ride = costliest_ride['haversine_dist'].values[0]

print("Haversine Distance for the Costliest Ride:",
      haversine_dist_costliest_ride, "kilometers")





Haversine Distance for the Costliest Ride: 0.0007899213191009993 kilometers


In [14]:
df_filtered = df[df['pickup_datetime'].dt.year == 2014]


number_of_rides = len(df_filtered)

print("The number of rides in 2014 is:", number_of_rides)

The number of rides in 2014 is: 29968


In [15]:
f_filtered = df[
    (df['pickup_datetime'].dt.year == 2014)
    & (df['pickup_datetime'].dt.quarter == 1)
]

# Get the number of rides
number_of_rides = len(df_filtered)

print("The number of rides in the first quarter of 2014 is:", number_of_rides)

The number of rides in the first quarter of 2014 is: 29968


In [16]:
ride_sept_2010 = df[(df['pickup_datetime'].dt.year == 2010) & (df['pickup_datetime'].dt.month == 9)]

rides_by_day = ride_sept_2010.groupby(ride_sept_2010['pickup_datetime'].dt.day_name()).size()


max_rides_day = rides_by_day.idxmax()
max_rides_count = rides_by_day.max()

print("On which day of the week in September 2010, maximum rides were recorded?")
print("Day:", max_rides_day)
print("Number of rides:", max_rides_count)

On which day of the week in September 2010, maximum rides were recorded?
Day: Thursday
Number of rides: 457


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error


In [18]:
df.columns

Index(['ride_id', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'haversine_dist', 'pickup_lat_rad', 'pickup_lon_rad',
       'dropoff_lat_rad', 'dropoff_lon_rad', 'dlat', 'dlon', 'a', 'c'],
      dtype='object')

In [19]:
df['year'] = df['pickup_datetime'].dt.year
df['month'] = df['pickup_datetime'].dt.month
df['week'] = df['pickup_datetime'].dt.week

<ipython-input-19-262df957df9c>:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week'] = df['pickup_datetime'].dt.week


In [20]:
y = df['fare_amount']

In [21]:
x=df[['passenger_count','haversine_dist','week']]

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.70, random_state=100)

In [23]:
x_train

,passenger_count,haversine_dist,week
195896,3,2.034021,39
47114,2,3.987687,33
144876,2,1.313580,23
184791,2,0.780877,20
123270,1,0.805125,39
...,...,...,...
194443,5,1.557800,38
65615,2,2.610860,43
77655,1,0.078202,25
56088,1,3.386177,48


In [24]:
x_train_num = x_train.select_dtypes(include=['int64', 'float64'])
x_train_num

,passenger_count,haversine_dist,week
195896,3,2.034021,39
47114,2,3.987687,33
144876,2,1.313580,23
184791,2,0.780877,20
123270,1,0.805125,39
...,...,...,...
194443,5,1.557800,38
65615,2,2.610860,43
77655,1,0.078202,25
56088,1,3.386177,48


In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_num_rescaled = pd.DataFrame(scaler.fit_transform(x_train_num),
                                    columns = x_train_num.columns,
                                    index = x_train_num.index)

x_train_num_rescaled.head()

,passenger_count,haversine_dist,week
195896,1.008439,-0.049696,0.900359
47114,0.242972,-0.044699,0.499309
144876,0.242972,-0.051539,-0.169108
184791,0.242972,-0.052902,-0.369633
123270,-0.522495,-0.052840,0.900359


In [26]:
x_test_num = x_test.select_dtypes(include=['int64', 'float64'])
x_test_num

,passenger_count,haversine_dist,week
180481,1,0.996834,48
8971,1,2.537148,13
77027,5,3.716854,21
21656,3,4.264505,33
58303,1,3.235923,9
...,...,...,...
188899,2,1.137244,49
98650,1,8666.117300,10
538,1,0.646667,31
60387,1,1.833666,4


In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_test_num_rescaled = pd.DataFrame(scaler.fit_transform(x_test_num),
                                    columns = x_test_num.columns,
                                    index = x_test_num.index)

x_test_num_rescaled.head()

,passenger_count,haversine_dist,week
180481,-0.442889,-0.050712,1.506695
8971,-0.442889,-0.046476,-0.834890
77027,2.127876,-0.043232,-0.299671
21656,0.842493,-0.041725,0.503159
58303,-0.442889,-0.044554,-1.102500


In [31]:
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'KNN Regression': KNeighborsRegressor()
}

In [32]:
adjusted_r2_scores = {}

for model_name, model in models.items():
    model.fit(x_train_num_rescaled, y_train)
    y_pred = model.predict(x_test_num_rescaled)


    r2 = r2_score(y_test, y_pred)


    n = x_test_num_rescaled.shape[0]
    p = x_test_num_rescaled.shape[1]
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

    adjusted_r2_scores[model_name] = adjusted_r2

In [33]:
bad_model = min(adjusted_r2_scores, key=adjusted_r2_scores.get)

print("Model with the least adjusted R-squared:", bad_model)

Model with the least adjusted R-squared: Linear Regression
